In [1]:
import numpy as np

In [2]:
def convolution2d(input_tensor, filters, stride=1, padding=0):
    h, w, c = input_tensor.shape
    num_filters, filter_h, filter_w, _ = filters.shape
    
    padded_input = np.pad(input_tensor, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
    
    output_h = (h + 2 * padding - filter_h) // stride + 1
    output_w = (w + 2 * padding - filter_w) // stride + 1
    output_tensor = np.zeros((output_h, output_w, num_filters))
    
    for f in range(num_filters):
        for i in range(output_h):
            for j in range(output_w):
                h_start = i * stride
                h_end = h_start + filter_h
                w_start = j * stride
                w_end = w_start + filter_w
                
                output_tensor[i, j, f] = np.sum(padded_input[h_start:h_end, w_start:w_end, :] * filters[f])

    return output_tensor

In [3]:
def depthwise_convolution(input_tensor, depthwise_filters, stride=1, padding=0):
    h, w, c = input_tensor.shape
    num_channels, filter_h, filter_w = depthwise_filters.shape
    
    assert c == num_channels
    
    padded_input = np.pad(input_tensor, ((padding, padding), (padding, padding), (0, 0)), mode='constant')
    
    output_h = (h + 2 * padding - filter_h) // stride + 1
    output_w = (w + 2 * padding - filter_w) // stride + 1
    output_tensor = np.zeros((output_h, output_w, num_channels))
    
    for c in range(num_channels):
        for i in range(output_h):
            for j in range(output_w):
                h_start = i * stride
                h_end = h_start + filter_h
                w_start = j * stride
                w_end = w_start + filter_w
                
                output_tensor[i, j, c] = np.sum(padded_input[h_start:h_end, w_start:w_end, c] * depthwise_filters[c])

    return output_tensor

def pointwise_convolution(input_tensor, pointwise_filters):
    h, w, c = input_tensor.shape
    num_output_channels, _ = pointwise_filters.shape
    
    output_tensor = np.zeros((h, w, num_output_channels))
    
    for out_c in range(num_output_channels):
        for i in range(h):
            for j in range(w):
                output_tensor[i, j, out_c] = np.sum(input_tensor[i, j, :] * pointwise_filters[out_c])
    
    return output_tensor

def depthwise_separable_convolution(input_tensor, depthwise_filters, pointwise_filters, stride=1, padding=0):
    depthwise_output = depthwise_convolution(input_tensor, depthwise_filters, stride, padding)
    output = pointwise_convolution(depthwise_output, pointwise_filters)
    return output

In [8]:
input_tensor = np.random.rand(5, 5, 3)
stride = 1
padding = 1

In [11]:
depthwise_filters = np.random.rand(3, 3, 3)
pointwise_filters = np.random.rand(2, 3)

def test_dsc():
    output = depthwise_separable_convolution(input_tensor, depthwise_filters, pointwise_filters, stride, padding)
    return output.shape
print(test_dsc())

(5, 5, 2)


In [13]:
filters = np.random.rand(2, 3, 3, 3)

def test_c2d():
    output = convolution2d(input_tensor, filters, stride, padding)
    return output.shape
print(test_c2d())

(5, 5, 2)


In [14]:
import timeit

time_c2d = timeit.timeit(test_c2d, number=1000)
print(f"Время выполнения convolution 2d: {time_c2d}s")
time_dsc = timeit.timeit(test_dsc, number=1000)
print(f"Время выполнения depthwise separable convolution: {time_dsc}s")

Время выполнения convolution 2d: 0.1987787660000322s
Время выполнения depthwise separable convolution: 0.34832910699969943s
